# M02E10: Machine Learning Exercises 1 - Sample Solutions 

---

## **Task 1** - Working with Databases in Python

Assume that the HR department of a company is requested to generate reports to better understand their departments and employees. Your job as a Data Scientist is to pose SQL queries to their employees database to answer the questions below. 
**Note**: In this task you will use the **MySQL Employees sample database**. This database includes six separate tables and consists of 4 million records in total. You can check how its schema looks like [here](https://dev.mysql.com/doc/employee/en/sakila-structure.html).

1. What is the number of employees working with the company?
2. What are the smallest departments in number of employees?
3. What is the total payroll of each department? 
4. What is the average salary paid in each department? 
5. What are the job titles with the highest and the lowest salaries?
6. What is the number of employees with the same job title?
7. What is the number of employees that earn more than $100000/year?
8. What is the overall cost of giving a 10 percent raise to all employees in the company?

**Solution sketch**

1. Open a connection to MySQL, specifying the employees database.
2. Use the schema information provided in the link above to create appropriate SQL queries.
3. Check your results using Pandas. You can reproduce query results using a DataFrame method like `.query()`, `.loc()`, `.join()`,  or `.groupby()`.

**Sample solution**

In [ ]:
import mysql.connector as mysql
import pandas as pd

dbconnection = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "vossensql",
    database="employees"
)

### 1. What is the number of employees working with the company?

In [ ]:
pd.read_sql('SELECT COUNT(DISTINCT emp_no) AS empcount FROM employees', dbconnection)

### 2. What are the smallest departments in number of employees?

In [ ]:
# List all departments ordered by its number of employees
pd.read_sql(
    '''
    SELECT dept_no as dept, COUNT(emp_no) AS empcount 
    FROM dept_emp GROUP BY dept ORDER BY empcount ASC
    ''', 
    dbconnection
)

In [ ]:
# We nest the previous query to have the department with the minimum number of employees
pd.read_sql(
    '''
    SELECT dept, MIN(empcount) FROM 
    (SELECT dept_no AS dept, COUNT(emp_no) AS empcount 
    FROM dept_emp GROUP BY dept ORDER BY empcount ASC) AS countemp
    ''', 
    dbconnection
)

In [ ]:
# Alternatively, we could set a limit 1 to our first query
pd.read_sql(
    '''
    SELECT dept_no AS dept, COUNT(emp_no) AS empcount 
    FROM dept_emp GROUP BY dept ORDER BY empcount ASC LIMIT 1
    ''', 
    dbconnection
)

### 3. What is the total payroll of each department?

In [ ]:
# First, we select only the most up-to-date salary paid to an employee (nested query) 
# Then, we join our nested query with dept_emp to find out in which departments employees work for
# Finally, we group results by dept_no and sum salaries
pd.read_sql(
    '''
    SELECT dept_no AS dept, SUM(salary) AS payroll 
    FROM dept_emp
    INNER JOIN (SELECT emp_no AS emp, max(to_date) AS maxdate, salary FROM salaries GROUP BY emp_no) AS s ON  dept_emp.emp_no = s.emp
    GROUP BY dept_emp.dept_no
    ORDER BY payroll ASC
    ''', 
    dbconnection
)

### 4. What is the average salary paid in each department?

In [ ]:
# We use the previous query and change only 
# the AVG() function instead of SUM()
pd.read_sql(
    '''
    SELECT dept_no AS dept, AVG(salary) AS payroll 
    FROM dept_emp
    INNER JOIN (SELECT emp_no AS emp, max(to_date) AS maxdate, salary FROM salaries GROUP BY emp_no) AS s ON  dept_emp.emp_no = s.emp
    GROUP BY dept_emp.dept_no
    ORDER BY payroll ASC
    ''', 
    dbconnection
)

###  5. What are the job titles with the highest and the lowest salaries?

In [ ]:
# First, we join the tables salaries and titles.
# Then, we select job titles whose salaries are equal either to the max salary or to the min salary.
# Finally, we group results by job title.
pd.read_sql(
    '''SELECT s.salary, t.title FROM salaries s, titles t
        WHERE s.emp_no = t.emp_no AND (salary = (SELECT MAX(salary) FROM salaries) 
        OR salary = (SELECT MIN(salary) FROM salaries))
        GROUP BY title;''', 
    dbconnection
)

### 6. What is the number of employees with the same job title?

In [ ]:
pd.read_sql(
    '''SELECT COUNT(t.emp_no) AS employees_count, t.title AS job_title FROM titles t
        GROUP BY job_title
        ORDER BY employees_count DESC;''', 
    dbconnection
)

### 7. What is the number of employees that earn more than $100000/year?

In [ ]:
pd.read_sql(
    '''SELECT COUNT(emp_no) AS employees_earning_more_100000 FROM salaries
        WHERE salary > 100000
        ''', 
    dbconnection
)

### 8. What is the overall cost of giving a 10 percent raise to all employees in the company?

In [ ]:
# The first SUM() is used to compute the current payroll.
# The second is used to compute the payroll with a 10 percent rise.
# The third is used to compute the cost (the "damage" to the company).
# Note that we filter salaries by date, assuming that we are interest in
# an estimate starting from year 2000 (since the data doesn't include 2021).
pd.read_sql(
    '''SELECT SUM(salary) AS current_payroll, 
        SUM(salary+(salary*10/100)) AS payroll_plus_10pc, 
        SUM(salary) - SUM(salary+(salary*10/100)) AS damage 
        FROM salaries WHERE to_date > '2000-01-01'
        ''', 
    dbconnection
)

In [ ]:
dbconnection.close()

## **Task 2** - Market Basket Analysis

UMall is a grocery store that wants to achieve a competitive advantage in the market by exploiting the value of the data collected from product purchases. The company hired you as **Data Scientist** and provided you with a dataset file **umall.csv** (check your Data directory) including purchase transactions made in the store. Your task is to analyze the data to address the following questions UMall is interested in:

1. What are the 25 most purchased items? Can you describe any insightful pattern in regarding food choices (e.g., junk food *versus* green food) of their customers? Plot graphics to support your findings.

2. What is the probability of each of the 25 most purchased items to appear in a customer's basket?

3. Can we recommend items to customers based on their purchase patterns? Apply the **Apriori algorithm** to find association rules that can be used as recommendations in the future. Which meaningful and/or strange patterns you could find?

4. Discuss how seasonality bias (e.g., a high frequency of Christmas items being purchased in December) in the data can compromise Market Basket Analysis and how to overcome this problem.

**Note**: The dataset used here was extracted from the [Kaggle](https://www.kaggle.com/roshansharma/market-basket-optimization). Check the page for hints on how to proceed with the analysis.

**Solution sketch**

1. Load the dataset with Pandas.
2. Use a One-Hot encoding to create a better data representation.
3. Use `.sum()` and `.sort_values()` to find the most frequent items. Probabilities can be computed likewise.
4. Use `seaborn` or `matplotlib` to create plots.
5. Use the `mlxtend` library implementation of Apriori. 

**Sample solution**

In [ ]:
%matplotlib inline

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

plt.style.use('seaborn') # Configure plt to use seaborn plot style

In [ ]:
dataset = pd.read_csv(os.path.join('Data', 'umall.csv'), header=None)

In [ ]:
transactions = []

# Convert items in the dataframe to string
for row in dataset.values:
    transactions.append([str(item) for item in row if str(item) != 'nan']) # Remove missing values

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

### 1. What are the 25 most purchased items? Can you describe any insightful pattern in regarding food choices (e.g., junk food *versus* green food) of their customers? Plot graphics to support your findings.

In [ ]:
# 1. Use the sum() method to find item frequencies.
# 2. Sort frequencies using sort_values()
# 3. Select the top 25 products using .head(25)
top25 = df.sum().sort_values(ascending=False).head(25)
top25

In [ ]:
custom_fontsize = 20
fig, ax = plt.subplots(figsize=(20, 8)) # Set figure size
plt.title('25 most popular items', fontsize=custom_fontsize)
plt.xlabel('items', fontsize=custom_fontsize)
plt.ylabel('purchase frequency', fontsize=custom_fontsize)

plt.xticks(ha='left', rotation=-30, fontsize=custom_fontsize) # Rotate x-lables
plt
sns.barplot(x=top25.index, y=top25.values, palette='plasma') # Create a bar chart. Set colormap to "plasma"
plt.show()

**Insights**: 

* Junk food is on the top sales: The top 5 most common items in baskets are mineral watter, eggs, spaghetti, french fries and chocolate.
* The first "green food" to appear is frozen vegetables with purchase frequency around 750.

### 2. What is the probability of each of the 25 most purchased items to appear in a customer's basket?

In [ ]:
# In this market basket analysis, the probability of an item or itemset 
# appearing in a transaction is called support.

support_df = pd.DataFrame(top25/df.shape[0], columns=['Support'])
support_df

**Insights**:

* The probability to have a mineral water in a random basket is ~24%.
* The probability to have a eggs in a random basket is ~17%.

### 3. Can we recommend items to customers based on their purchase patterns? Apply the **Apriori algorithm** to find association rules that can be used as recommendations in the future. Which meaningful and/or strange patterns you could find?

In [ ]:
# We extract frequent itemsets mlxtend
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

To extract association rules, we use the [lift metric](https://en.wikipedia.org/wiki/Lift_(data_mining)). Lift measures how good an association rule is at classifying cases against a random choice. Lift = 1 means that the probability of occurrence of the antecedent and that of the consequent are independent of each other.

If the lift is > 1, it represents the degree to which occurrences are dependent on one another, and makes those rules potentially useful for predicting the consequent in future data sets.

In [ ]:
association_rules(frequent_itemsets, metric='lift', min_threshold=2.5)

According to above table, we can easily say that the dependency between **herb & pepper** and **ground beef** is high since lift score is around 3.3 and the confidence score is promising (~32%). Hence, it would be better to keep them close to increase sales!

### 4. Discuss how seasonality bias (e.g., a high frequency of Christmas items being purchased in December) in the data can compromise Market Basket Analysis and how to overcome this problem.


The insights above are completely data-driven and, therefore, should be interpreted carefully. For instance, if the data refers to purchases made in December, we should consider a bias towards Christmas items, e.g., turkey could have a higher proportion because of the period. In practice, grouping data by similar months help us to consider the seasonality of items, which may provide additional insights (e.g., desing offers targeted to specific holidays).

## **Task 3** - Drug Classification

The University Hospital has perceived a recent influx of patients that has led to increasingly hectic schedules for the healthcare staff. After observing the hospital's activity for a period of time, the doctors have noticed that many patients are coming in for similar health concerns and the vast majority of the prescribed medications are from a selection of 5 drugs. As a data scientist working for the hospital, your task is to construct a **K-nearest neighbors (KNN) classifier** based on the **drug200.csv** file (included in the Data directory). The classifier should predict which drug should be prescribed given the patient data.

If the classifier works well, patient hospital visits can be more streamlined with a physician at the end for checking to ensure the prescribed medication is correct. The hospital hopes that this will alleviate some pressure on the staff while maintaining the hospital's ability to treat patients.

**Note**: The dataset used was extracted from [Kaggle](https://www.kaggle.com/prathamtripathi/drug-classification). Feel free to have a look at the available notebooks to find some hints that may help you to accomplish this task. Additionally, we provide you with a short explanation about KNN below.

### K-nearest neighbors - a lazy learning algorithm (see M2E10)

The KNN algorithm itself is fairly straightforward and can be summarized by the following steps:

1. Choose the number of k and a distance metric.
2. Find the k-nearest neighbors of the data record that we want to classify.
3. Assign the class label by majority vote.

The following figure illustrates how a new data point, indicated by ?, is assigned the triangle class label based on majority voting among its five nearest neighbors:

<div style='align: left; text-align:center;'><br/>
    <img src='img/knnprinc.png' alt='KNN principle' style="height:250px;"/>
    <span style='display:block;'>KNN Principle. Image Source: <a href="https://subscription.packtpub.com/book/data/9781789955750/2/ch02lvl1sec09/artificial-neurons-a-brief-glimpse-into-the-early-history-of-machine-learning" target="_blank">The Neural Network Blog</a>.</span>
    <br/>
</div>

Based on the chosen distance metric, the KNN algorithm finds the k examples in the training dataset that are closest (most similar) to the point that we want to classify. The class label of the data point is then determined by a majority vote among its k nearest neighbors.

The main advantage of such a "memory-based" approach is that the classifier immediately adapts as we collect new training data. However, the downside is that the computational complexity for classifying new examples grows linearly with the number of examples in the training dataset in the worst-case scenario—unless the dataset has very few dimensions (features) and the algorithm has been implemented using efficient data structures. Furthermore, we can't discard training examples since no training step is involved. Thus, storage space can become a challenge if we are working with large datasets.

The right choice of k is crucial to finding a good balance between overfitting and underfitting. We also have to make sure that we choose a distance metric that is appropriate for the features in the dataset. Often, a simple Euclidean distance measure is used for real-value examples, for example, the flowers in the Iris dataset, which have features measured in centimeters. However, if we are using a Euclidean distance measure, it is also important to standardize the data so that each feature contributes equally to the distance.

**Solution sketch**

1. Load the dataset with Pandas.
2. Use `from sklearn.preprocessing import OrdinalEncoder` to transform ordinal categorical features into a numerical representation. Additionally, handle non-ordinal categorical features with one-hot encoding.
3. Use `from sklearn.preprocessing import LabelEncoder` to transform the class label into a numerical representation.
4. Split the preprocessed data into train/test. Since the data is imbalanced, you should use a [stratified shuffle split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
5. Select the best value of k in between 1 and 20 (inclusive).
6. Use `from sklearn.metrics import plot_confusion_matrix` to plot the confusion matrix for the best KNN model found.

**Sample solution**

In [ ]:
%matplotlib inline

import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import plot_confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier

plt.style.use('seaborn') # Configure plt to use seaborn plot style
sns.set(font_scale=1.2)  # Set the font size for all plots

### Load the data 

We start by loading the data and checking data types for each column.

In [ ]:
# Load the dataset
drug_data = pd.read_csv(os.path.join('Data', 'drug200.csv'))
drug_data.head()

In [ ]:
# Check data types
drug_data.info()

### Exploratory Data Analysis

Let's check how the features affect drug type prescriptions. Here, interesting questions to ask the data are: 

* Are there certain threshold values that lead to drug X prescribed over drug Y? 
* What are the combinations of feature (e.g., "Age" and "Na_to_K") values, if any, that lead to drug Z to be prescribed over drug W? 

These are all questions we can investigate by plotting each feature against "Drug".

In [ ]:
# Checking the relationship between Drug and Sex
sns.countplot(x="Drug", hue="Sex", data=drug_data)

plt.title("Sex vs. Drug")
plt.xlabel("Drug")
plt.ylabel("Total Counts")
plt.show()

**Findings**: There doesn't seem to be a drug that is prescribed exclusively for each sex. In fact, the distributions look quite similar, suggesting that "Sex" is not very important in determining the drug type.

In [ ]:
# Checking the relationship between Drug and Age
sns.catplot(x="Age", y="Drug", data=drug_data)
plt.title("Age vs. Drug")
plt.show()

**Findings**: Drugs Y, C, and X seem to be prescribed for all ages in the dataset while A and B have clear age cut-offs. It looks like drug A is precribed for ages ~ [20-50] while B is prescribed for ages ~ [51-75]. Our classifier should take this into account.

In [ ]:
# Checking the relationship between Drug and BP
sns.countplot(x="Drug", hue="BP", data=drug_data)
plt.title("BP vs Drug")
plt.xlabel("Drug")
plt.ylabel("Total Counts")
plt.legend(loc='upper right') # Position legend
plt.show()

**Findings**: BP levels seem to affect which drug type is prescribed. 

* "High" BP patients were prescribed drugs B, A, and Y. 
* "Low" BP patients were prescribed drugs X, C, and Y. 
* "Normal" BP patients were prescribed drugs X and Y.

Notice that some drugs were prescribed to multiple BP levels (e.g., drug Y: "High", "Low", and "Normal"). In contrast, some drugs were prescribed exclusively to a specific BP level (e.g., drug C: "Low"). Hence, BP seems to be informative but not deterministic in deciding which drug type to prescribe.

In [ ]:
# Checking the relationship between Drug and Cholesterol
sns.countplot(x="Drug", hue="Cholesterol", data=drug_data)
plt.title("Cholesterol vs. Drug")
plt.xlabel("Drug")
plt.ylabel("Total Counts")
plt.show()

**Findings**: It looks like most drugs are prescribed to patients with both cholesterol levels. The only exception is drug C which is exclusively prescribed to patients with "High" cholesterol.

In [ ]:
# Checking the relationship between Drug and Na_to_K
sns.catplot(x="Na_to_K", y="Drug", data=drug_data)
plt.title("Na_to_K vs. Drug")
plt.show()

**Findings**: It looks like drugs C, X, A, and B are prescribed for patients with "Na_to_K" values below 15 while drug Y is exclusively precribed for values above.

**Overall insight (for feature selection)**: We can use our findings to select features that can discriminate different drug prescriptions. For instance, we notice that drug Y was prescribed for all values (e.g., "all ages" and "High", "Low", and "Normal" BP). However, in the "Na_to_K" plot above, only patients with values greater than 15 are prescribed drug Y. Therefore, "Na_to_K" measurements seem to be crucial in deciding whether to prescribe drug Y or not. On the other hand, feature "Sex" has no apparent impact on the drug prescrition and can be removed from our feature set. Our classifier should take that into consideration.

### Data preparation

Some features contain categorical values (e.g., "BP" with values "High", "Low", and "Normal") and must be transformed into a numerical representation.

We start by transforming ordinal features:

In [ ]:
ordinal_enc = OrdinalEncoder()

ordinal_features = drug_data[["BP", "Cholesterol"]]
encoded_ordinal = ordinal_enc.fit_transform(ordinal_features)

Now, let's transform class labels (i.e., prescribed drugs):

In [ ]:
label_enc = LabelEncoder()
# Reshape is needed for sklearn classifiers
encoded_labels = label_enc.fit_transform(drug_data.Drug).reshape(-1,1)

We can create a new dataset that contains all encoded features and lables:

In [ ]:
# New DataFrame with encoded features
ordinal_df = pd.DataFrame(encoded_ordinal, columns=["BP", "Cholesterol"])

# New DataFrame with encoded lables
labels_df = pd.DataFrame(encoded_labels, columns=["Drug"])

# Make a copy of the original DataFrame without the categorical features.
# Note that we remove the "Sex" feature, because it seems to be uninformative.
numeric_df = drug_data.drop(["BP", "Cholesterol", "Drug", "Sex"], axis=1)

# Concat all features with lables
prep_dataset = pd.concat([numeric_df, ordinal_df, labels_df], axis=1)

prep_dataset.head()

Let's separate features from labels:

In [ ]:
X = prep_dataset[prep_dataset.columns[:-1]]
y = prep_dataset.Drug

Let's scale the features to avoid giving more importance to features with a larger scale:

In [ ]:
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)

We generate train/test sets using a stratified sampling to account label imbalance:

In [ ]:
# Random_state=42 is used for reproducibility.
# It guarantees that the split will be the same every time.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

### Data Modeling

With the dataset ready for modeling, we can start defining and training our KNN estimator/model:

In [ ]:
estimator = KNeighborsClassifier(n_neighbors=5)
estimator.fit(X_train, y_train)

**Evaluating training performance**

In [ ]:
print(classification_report(y_train, estimator.predict(X_train)))

**Evaluating test performance**

In [ ]:
print(classification_report(y_test, estimator.predict(X_test)))

**Confusion Matrix**

In [ ]:
plot_confusion_matrix(estimator, X, y, cmap="Blues")
plt.show()

**Model Selection**

Let's use a StratifiedKFold cross-validation procedure to select the best number of k within the range \[1, 20\]:

In [ ]:
values_of_k = []
accuracy = []

for k in range(1, 21):
    # Create estimator
    estimator = KNeighborsClassifier(n_neighbors=k)
    # Define the evaluation procedure
    cv = StratifiedKFold(n_splits=5)
    # Compute the cross-validation test accuracy score
    scores = cross_val_score(estimator, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    # Compute mean accuracy
    accuracy.append(np.mean(scores))
    # Add the current value of k to create a plot.
    values_of_k.append(k)

plt.title('Impact of k') # Define the plot title
plt.plot(values_of_k, accuracy) # Plot the accuracy score produced by each value of k
plt.xticks([i for i in range(1, 21)])
plt.yticks(np.linspace(0.6, 0.9, 10))
plt.xlim(1) # Set the x-axis at 1. The dafault value is 0.
plt.show()

---

# End M2E10: Machine Learning Exercises 1 - Sample Solutions 